In [64]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from data_processing import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from data_transformar import *
import lightgbm as lgb
from sklearn.decomposition import PCA
from catboost import CatBoostRegressor, Pool

warnings.simplefilter("ignore")
%matplotlib inline
%load_ext autoreload
%autoreload 2

sns.set(style="darkgrid")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from sklearn.metrics import mean_squared_error

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [2]:
data = pd.read_csv("./train.csv")
data = data.drop(columns=["Id"])

In [3]:
y = np.log1p(data["SalePrice"])
X = data.drop(columns=["SalePrice"])

print(X.shape)
X.head()

(1460, 79)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=98987)

In [79]:
transformer = DataTransformer(StandardScaler())

In [80]:
transformer.nums_to_cats(X_train)
transformer.nums_to_cats(X_test)

transformer.fillna(X_train)
transformer.fillna(X_test)

num_candidates = list(X_train.dtypes[X_train.dtypes!="object"].index.values)
cat_candidates = list(X_train.dtypes[X_train.dtypes=="object"].index.values)

cats_to_drop = ["Utilities"]
X_train = X_train.drop(cats_to_drop, axis=1)
X_test = X_test.drop(cats_to_drop, axis=1)

In [81]:
n_cols = X_train.select_dtypes([np.number]).columns

transformer.imputer_fit(X_train[n_cols])

X_train[n_cols] = transformer.imputer_transform(X_train[n_cols])
X_test[n_cols] = transformer.imputer_transform(X_test[n_cols])

transformer.scaler_fit(X_train[n_cols])

X_train[n_cols] = transformer.scaler_transform(X_train[n_cols])
X_test[n_cols] = transformer.scaler_transform(X_test[n_cols])

In [82]:
X_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1079,20,RL,-0.246209,-0.188899,Pave,None,Reg,Lvl,Inside,Gtl,...,-0.2627,-0.071012,None,None,None,-0.082033,4,2007,WD,Normal
389,60,RL,1.128006,0.222697,Pave,None,Reg,Lvl,Inside,Gtl,...,-0.2627,-0.071012,None,None,None,-0.082033,8,2008,New,Partial
1349,70,RM,-0.911152,-0.581133,Pave,Pave,Reg,Lvl,Inside,Gtl,...,-0.2627,-0.071012,None,None,None,-0.082033,12,2008,WD,Normal
1093,20,RL,0.019768,-0.138270,Pave,None,Reg,Lvl,Corner,Gtl,...,-0.2627,-0.071012,None,MnPrv,None,-0.082033,10,2006,WD,Normal
740,70,RM,-0.467857,-0.097099,Pave,Grvl,Reg,Lvl,Inside,Gtl,...,-0.2627,-0.071012,None,GdPrv,None,-0.082033,5,2007,WD,Abnorml


In [83]:
org_params = {
    'iterations': 10000,
    'learning_rate': 0.08,
    'eval_metric': 'RMSE',
    'random_seed': 42,
    'logging_level': 'Silent',
    'use_best_model': True,
    'loss_function': 'RMSE',
    'od_type': 'Iter',
    'od_wait': 1000,
    'one_hot_max_size': 20,
    'l2_leaf_reg': 100,
    'depth': 3,
    'rsm': 0.6,
    'random_strength': 2,
    'bagging_temperature': 10
}

In [84]:
cat_features = np.where(X_train.loc[:, X_train.columns.values].dtypes=="object")[0]

train_pool = Pool(X_train, y_train, cat_features=cat_features)
dev_pool = Pool(X_test, y_test, cat_features=cat_features)

model = CatBoostRegressor(**org_params)
model.fit(train_pool, eval_set=dev_pool, plot = False)

CatBoostError: Invalid type for cat_feature[non-default value idx=0,feature_idx=57]=1996.0 : cat_features must be integer or string, real number values and NaN values should be converted to string.

In [90]:
X_train.iloc[:, 57]

1079    1996.0
389     2008.0
1349      None
1093    1977.0
740     1910.0
         ...  
133     2001.0
1288    2000.0
180     2000.0
1290    1964.0
981     1998.0
Name: GarageYrBlt, Length: 1168, dtype: object